In [11]:
import requests
import json
import pandas as pd

from mbuskey import API_KEY

In [7]:
ENDPOINT = "https://mbus.ltp.umich.edu/bustime/api/v3"

In [8]:
def get_bus_routes(params={"key": API_KEY, "format": "json"}):
    """Get all bus routes"""
    response = requests.get(f"{ENDPOINT}/getroutes", params=params)
    return response.json()

In [9]:
def get_vehicles(rt):
    """Get all active vehicles"""
    params = {"key": API_KEY, "format": "json", "rt": rt}
    response = requests.get(f"{ENDPOINT}/getvehicles", params=params)
    if response.json()["bustime-response"].get("error"):
        return []
    return [bus for bus in response.json()["bustime-response"].get("vehicle")]

In [12]:
pd.DataFrame(get_vehicles("CS"))

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,spd,tatripid,origtatripno,tablockid,zone,mode,psgld,stst,stsd
0,3040,20231127 15:51,42.28909156849836,-83.71972328722659,161,12,CS,Lot SC-7,6534,False,33,488172,488172,110COM,,0,FULL,56700,2023-11-27
1,3041,20231127 15:51,42.27888634172821,-83.7371148351346,178,12,CS,Lot SC-7,18685,False,0,488173,488173,116COM,,0,FULL,56100,2023-11-27


In [5]:
def check_delays(bus_list):
    """Check if any buses are delayed"""
    for bus in bus_list:
        if bus["dly"] == "true":
            return True
    return False

In [6]:
def get_route_name(rt):
    for route in get_bus_routes()["bustime-response"]["routes"]:
        if route["rt"] == rt:
            return route["rtnm"]

In [7]:
def user_delay_checker():
    routes = get_bus_routes()["bustime-response"]["routes"]
    all_buses = []
    for route in routes:
        for bus in get_vehicles(route["rt"]):
            all_buses.append(bus)

    current_rts = []
    for bus in all_buses:
        if bus["rt"] not in current_rts:
            current_rts.append(bus["rt"])

    current_rtnms = [get_route_name(rt) for rt in current_rts]

    print("--- Current Routes ---")
    for rtnm in current_rtnms:
        print(rtnm)
    print("----------------------")
    while True:
        user_route = input("Which route would you like to check for delays? ")

        for route in routes:
            if route["rtnm"] == user_route:
                user_rt = route["rt"]

        user_rt_delay = check_delays(get_vehicles(user_rt))

        if user_rt_delay:
            print()
            print(f"{user_route} is delayed!")
            print()
        else:
            print()
            print(f"{user_route} is not delayed!")
            print()

        user_continue = input("Would you like to check another route? (y/n) ")
        if user_continue == "n":
            print()
            print("Thanks for checking the buses!")
            print()
            break

In [8]:
user_delay_checker()

--- Current Routes ---
Bursley Baits
Commuter North
Commuter South
Diag-to-Diag Express
Med Express
North-East Shuttle
Northwood
Wall Street-NIB
----------------------

Bursley Baits is not delayed!


Thanks for checking the buses!

